In [20]:
import tensorflow.compat.v1 as tf
import pandas as pd

In [21]:
tf.disable_eager_execution()
tf.disable_v2_behavior()

In [22]:
data = pd.read_csv("breast_cancer.csv")

In [23]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,116.0,57.301724,16.112766,24.000000,45.000000,56.000000,71.000000,89.000000
BMI,116.0,27.582111,5.020136,18.370000,22.973205,27.662416,31.241442,38.578759
Glucose,116.0,97.793103,22.525162,60.000000,85.750000,92.000000,102.000000,201.000000
Insulin,116.0,10.012086,10.067768,2.432000,4.359250,5.924500,11.189250,58.460000
HOMA,116.0,2.694988,3.642043,0.467409,0.917966,1.380939,2.857787,25.050342
Leptin,116.0,26.615080,19.183294,4.311000,12.313675,20.271000,37.378300,90.280000
Adiponectin,116.0,10.180874,6.843341,1.656020,5.474282,8.352692,11.815970,38.040000
Resistin,116.0,14.725966,12.390646,3.210000,6.881763,10.827740,17.755207,82.100000
MCP.1,116.0,534.647000,345.912663,45.843000,269.978250,471.322500,700.085000,1698.440000
Classification,116.0,1.551724,0.499475,1.000000,1.000000,2.000000,2.000000,2.000000


In [24]:
def label_fix(label):
    if label==1:
        return 0
    else:
        return 1

In [26]:
data['Classification'] = data['Classification'].apply(label_fix)

In [27]:
X = data.drop(['Classification'], axis=1)
y = data['Classification']

In [28]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
scalar = MinMaxScaler()

In [31]:
scalar.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [32]:
X_train = pd.DataFrame(data=scalar.transform(X_train),columns = X_train.columns,index=X_train.index)
X_test = pd.DataFrame(data=scalar.transform(X_test),columns = X_test.columns,index=X_test.index)

In [33]:
X_train.columns

Index(['Age', 'BMI', 'Glucose', 'Insulin', 'HOMA', 'Leptin', 'Adiponectin',
       'Resistin', 'MCP.1'],
      dtype='object')

In [34]:
age = tf.feature_column.numeric_column('Age')
bmi = tf.feature_column.numeric_column('BMI')
glucose = tf.feature_column.numeric_column('Glucose')
insulin = tf.feature_column.numeric_column('Insulin')
homa = tf.feature_column.numeric_column('HOMA')
leptin = tf.feature_column.numeric_column('Leptin')
adiponectin = tf.feature_column.numeric_column('Adiponectin')
resistin = tf.feature_column.numeric_column('Resistin')
mcp = tf.feature_column.numeric_column('MCP.1')

feat_cols = [ age, bmi, glucose, insulin, homa, leptin, adiponectin, resistin, mcp]

In [78]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train,y_train, batch_size=4, num_epochs=100, shuffle=True)
pred_func = tf.estimator.inputs.pandas_input_fn(X_test, batch_size=4, num_epochs=1, shuffle=False)

In [149]:
estimator = tf.estimator.DNNClassifier([7,7,7,7],feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfr_r4qij', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [150]:
estimator.train(input_func, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpfr_r4qij/model.ckpt.
INFO:tensorflow:loss = 3.0422766, step = 1
INFO:tensorflow:global_step/sec: 370.477
INFO:tensorflow:loss = 3.0521722, step = 101 (0.274 sec)
INFO:tensorflow:global_step/sec: 567.688
INFO:tensorflow:loss = 3.225821, step = 201 (0.172 sec)
INFO:tensorflow:global_step/sec: 516.207
INFO:tensorflow:loss = 2.103878, step = 301 (0.195 sec)
INFO:tensorflow:global_step/sec: 511.532
INFO:tensorflow:loss = 0.8032049, step = 401 (0.199 sec)
INFO:tensorflow:global_step/sec: 516.686
INFO:tensorflow:loss = 1.9735224, step = 501 (0.189 sec)
INFO:tensorflow:global_step/sec: 529.823
INFO:tensorflow:loss = 2.8883398, step = 601 (0.188 sec)
INFO:tensorflow:global_step/sec: 504.139
INFO:tensorflow:lo

In [151]:
predictions = list(estimator.predict(input_fn=pred_func))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfr_r4qij/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [152]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [153]:
from sklearn.metrics import classification_report

In [154]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.83      0.88      0.86        17
           1       0.88      0.83      0.86        18

    accuracy                           0.86        35
   macro avg       0.86      0.86      0.86        35
weighted avg       0.86      0.86      0.86        35

